In [ ]:
# Overview of geospatial data, coordinate systems and geometry types.  

In [ ]:
from sedona.spark import *

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)

In [ ]:
# Read and create basic data frames  - Pranav

sedona.read

In [ ]:
# Vector drivers  - Pranav

# GeoJSON
# CSV
# Shapefile

# Show others in comments

In [ ]:
# Raster - Pranav

# Raster

# Show others

In [ ]:
# Intro to cloud native formats - Pranav

# Demo the speed on this 
# COG from LANDSAT - STAC

In [ ]:
# Intro to Iceberg - Pranav

In [ ]:
# Transforming data with non-native readers - in slides

In [ ]:
# Wherobots Fundamentals - Constructing Geometries - Furqaan

In [ ]:
# Wherobots Fundamentals - Spatial predicates  - Furqaan

In [ ]:
# Wherobots Fundamentals - Range joins - Furqaan

In [ ]:
# create and manage Havasu (Iceberg) tables for vector and raster data  - Furqaan

In [ ]:
# Data validity checks - Matt

In [ ]:
# Handling and transforming CRS - Matt

In [ ]:
# Dataset loading - aka load all datasets to tables - Matt